## Setup and Import Libraries

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
llm = ChatGroq(model="Gemma2-9b-It")

## Prompt Template

In [5]:
system_message = """ You are a helpful assistant. Answer all the questions to the best of your ability
"""

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [7]:
chain = prompt | llm

In [8]:
chain.invoke({"messages": [HumanMessage(content='Hi My name is Saad')]})

AIMessage(content="Hi Saad, it's nice to meet you! \n\nHow can I help you today?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.002333545, 'queue_time': 0.017482805, 'total_time': 0.049606272}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aa6814eb-5d6a-4bb5-bcae-6be762ed40b4-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

## Message History with Prompt Template

In [9]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

In [10]:
config = {
    "configurable": {"session_id": "chat1"}
}

In [11]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [12]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist"),
    ],
    config=config
)

response.content

"Hello Saad Tariq, it's nice to meet you! \n\nI understand you're a Data Scientist. That's a fascinating field! \n\nIs there anything specific I can help you with today? Perhaps you have a data science problem you'd like to brainstorm, need information about a particular algorithm, or want to discuss the latest trends in the field? \n\nI'm here to assist in any way I can. 😊  \n\n"

## Adding More Input Variables

In [13]:
system_message = """ You are a helpful assistant. Answer all the questions to the best of your ability in {language}
"""

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [15]:
chain = prompt | llm

In [16]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content='Hi My name is Saad')],
        "language": "Urdu"
    }
)

response.content

'سلام! میرے نام کا تعلق اس ایڈی-ٹو-ٹوہی ہے. کیا میں آپ کی مدد کر سکتا ہوں؟   \n'

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config2 = {
    "configurable": {"session_id": "chat2"}
}

In [19]:
response = with_message_history.invoke(
    {
        'messages':[HumanMessage(content="Hi, My name is Saad Tariq")],
        'language': 'Urdu'
    },
    config=config2
)

response.content

' سلام! میرا نام بھی سعد تارق ہے، خوش آمدید! \n\n(Salam! My name is Saad Tariq as well, welcome!)\n\nکیا میں آپ کی کوئی مدد کر سکتا ہوں؟\n\n(Can I help you with anything?) \n'

In [20]:
response = with_message_history.invoke(
    {
        'messages':[HumanMessage(content="What is my name")],
        'language': 'Urdu'
    },
    config=config2
)

response.content

'اُس کا نام سعد تارق ہے\n\n(His name is Saad Tariq) \n'